In [1]:
%pip install pyspark==3.5.0 python-dotenv requests pandas

Note: you may need to restart the kernel to use updated packages.


In [4]:
import os
import sys
from pathlib import Path
from dotenv import load_dotenv
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

# 1. LOAD CREDENTIALS (Identical to Silver script)
# Since your notebook is in /notebooks, we go up one level to find .env
env_path = Path.cwd().parent / '.env'
load_dotenv(dotenv_path=env_path)

STORAGE_ACCOUNT = os.getenv("STORAGE_ACCOUNT")
CLIENT_ID       = os.getenv("CLIENT_ID")
TENANT_ID       = os.getenv("TENANT_ID")
CLIENT_SECRET   = os.getenv("CLIENT_SECRET")

# 2. BUILD THE SPARK SESSION
spark = SparkSession.builder \
    .appName("Aviation_EDA") \
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-azure:3.3.4,com.microsoft.azure:azure-storage:8.6.6") \
    .config(f"fs.azure.account.auth.type.{STORAGE_ACCOUNT}.dfs.core.windows.net", "OAuth") \
    .config(f"fs.azure.account.oauth.provider.type.{STORAGE_ACCOUNT}.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider") \
    .config(f"fs.azure.account.oauth2.client.id.{STORAGE_ACCOUNT}.dfs.core.windows.net", CLIENT_ID) \
    .config(f"fs.azure.account.oauth2.client.secret.{STORAGE_ACCOUNT}.dfs.core.windows.net", CLIENT_SECRET) \
    .config(f"fs.azure.account.oauth2.client.endpoint.{STORAGE_ACCOUNT}.dfs.core.windows.net", f"https://login.microsoftonline.com/{TENANT_ID}/oauth2/token") \
    .getOrCreate()

# 3. LOAD THE DATA
bronze_path = f"abfss://bronze@{STORAGE_ACCOUNT}.dfs.core.windows.net/raw_data"
df_raw = spark.read.json(bronze_path)

df_exploded = df_raw.select(F.explode("states").alias("s"))

# 3. Map the array indices to real names
# OpenSky uses fixed positions in the array: 0=icao24, 1=callsign, etc.
df_bronze = df_exploded.select(
    F.col("s")[0].alias("icao24"),
    F.col("s")[1].alias("callsign"),
    F.col("s")[2].alias("origin_country"),
    F.col("s")[5].alias("longitude"),
    F.col("s")[6].alias("latitude"),
    F.col("s")[7].alias("baro_altitude"),
    F.col("s")[9].alias("velocity"),
    F.col("s")[10].alias("true_track"),
    F.col("s")[11].alias("vertical_rate")
)
# 4. PREVIEW
df_bronze.limit(5).toPandas() # .toPandas() makes it look much nicer in Notebooks than .show()

,icao24,callsign,origin_country,longitude,latitude,baro_altitude,velocity,true_track,vertical_rate
0,80162d,AXB2547,India,73.5981,19.3104,4267.2,127.85,273.69,0
1,80162a,IGO7069,India,73.9691,18.0089,4282.44,102.22,172.48,1.3
2,e8027e,LXP252,Chile,-73.5653,-46.1019,10972.8,254.27,1.86,0.33
3,801638,AXB1520,India,87.3336,21.1503,11224.26,210.98,211.95,4.88
4,39de4a,TVF75MP,France,14.8972,45.0424,11567.16,185.67,283.46,-0.33


In [9]:
df_bronze.describe().toPandas()

26/02/14 07:47:51 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


,summary,icao24,callsign,origin_country,time_position,last_contact,long,lat,baro_altitude,velocity,true_track,vertical_rate,sensors,geo_altitude,squawk,position_source
0,count,200,200,200,195,200,195,195,178,199,200,184,0,178,104,200
1,mean,514514.23529411765,None,None,1.7709951214358974E9,1.7709951362E9,-26.998386153846155,36.86246974358975,7255.267415730337,162.53170854271355,191.2316,0.6518478260869567,None,7272.433820224719,3139.9326923076924,0.0
2,stddev,321084.1704387919,None,None,189.74967162773797,59.16062793881947,66.52661104865352,12.720917448177948,4220.833129486218,81.9411298969163,103.40150221219562,5.1347424641784984,None,4250.631861129824,2321.8043923212804,0.0
3,min,00b22b,,Austria,1770992637,1770994842,-135.1395,-33.115,-53.34,0.0,0.0,-15.28,None,7.62,0022,0
4,max,e8026f,WZZ9GN,United States,1770995156,1770995156,100.8535,66.9388,13716.0,290.21,359.61,20.48,None,13860.78,7660,0


In [11]:
df_bronze.dtypes

[('icao24', 'string'),
 ('callsign', 'string'),
 ('origin_country', 'string'),
 ('time_position', 'bigint'),
 ('last_contact', 'bigint'),
 ('long', 'double'),
 ('lat', 'double'),
 ('baro_altitude', 'double'),
 ('on_ground', 'boolean'),
 ('velocity', 'double'),
 ('true_track', 'double'),
 ('vertical_rate', 'double'),
 ('sensors', 'string'),
 ('geo_altitude', 'double'),
 ('squawk', 'string'),
 ('spi', 'boolean'),
 ('position_source', 'bigint')]

In [ ]:
from pyspark.sql.functions import col, count, when

#count nulls in each column
df_bronze.select([count(when(col(c).isNull(), c)).alias(c) for c in df_bronze.columns]).toPandas()

,icao24,callsign,origin_country,time_position,last_contact,long,lat,baro_altitude,on_ground,velocity,true_track,vertical_rate,sensors,geo_altitude,squawk,spi,position_source
0,0,0,0,5,0,5,5,22,0,1,0,16,200,22,96,0,0


In [12]:
#check for duplicates
total_count = df_bronze.count()
distinct_count = df_bronze.dropDuplicates(["icao24", "time_position"]).count()

print(f"Duplicate rows found: {total_count - distinct_count}")

Duplicate rows found: 0


In [13]:
#check time format
df_bronze.select("time_position").show(5)

+-------------+
|time_position|
+-------------+
|   1770995155|
|         NULL|
|   1770995156|
|   1770995155|
|   1770995155|
+-------------+
only showing top 5 rows

